In [10]:
import os
import subprocess
from datetime import datetime
from google.colab import drive

# === CẤU HÌNH RIÊNG (CHỈ THAY 1 LẦN) ===
USERNAME = "FongNgoo"   # ← THAY ĐỔI
REPO_NAME = "Basic_Dynamic_Prices_base_on_Demand_Model"        # ← THAY ĐỔI
BRANCH = "main"                     # hoặc "master"
# ======================================

REPO_DIR = f"/content/{REPO_NAME}"
REPO_URL = f"git@github.com:{USERNAME}/{REPO_NAME}.git"


In [11]:
def _setup_ssh():
    """Tải SSH key từ Drive và cấu hình"""
    drive.mount('/content/drive', force_remount=False)
    SSH_DIR = '/content/drive/MyDrive/.colab_ssh'
    SSH_KEY = f"{SSH_DIR}/id_colab"
    if not os.path.exists(SSH_KEY):
        raise FileNotFoundError("SSH key chưa tồn tại! Chạy file setup trước.")

    os.makedirs('/root/.ssh', exist_ok=True)
    subprocess.run(['cp', f'{SSH_KEY}', f'{SSH_KEY}.pub', '/root/.ssh/'], check=False)
    subprocess.run(['chmod', '600', '/root/.ssh/id_colab'], check=True)
    subprocess.run(['chmod', '644', '/root/.ssh/id_colab.pub'], check=True)

    config = f"""Host github.com
    HostName github.com
    User git
    IdentityFile /root/.ssh/id_colab
    StrictHostKeyChecking no
"""
    with open('/root/.ssh/config', 'w') as f:
        f.write(config)
    subprocess.run(['chmod', '600', '/root/.ssh/config'], check=True)

In [12]:
def _ensure_repo():
    """Đảm bảo repo tồn tại, clone nếu cần"""
    if not os.path.exists(REPO_DIR):
        print(f"Repo không tồn tại → Đang clone: {REPO_URL}")
        result = subprocess.run(['git', 'clone', REPO_URL, REPO_DIR])
        if result.returncode != 0:
            raise RuntimeError("Clone thất bại! Kiểm tra SSH key và repo URL.")
    # Luôn cd vào repo
    os.chdir(REPO_DIR)
    print(f"Đã vào thư mục: {REPO_DIR}")

In [17]:
def fix_ssh_and_push(message=None, force=False):
    """
    Hàm cứu cánh 1 lần gọi là xong mãi mãi:
    - Fix Drive treo
    - Tạo/tái tạo SSH key
    - Fix Host key verification failed
    - Push ngay lập tức (có force nếu cần)
    Gọi: fix_ssh_and_push()  → xong!
    """
    repo_path = "/content/drive/MyDrive/Colab_Notebooks/Basic_Dynamic_Prices_base_on_Demand_Model"

    # 1. Fix Drive treo nặng nhất có thể
    !pkill -9 -f "drive" 2>/dev/null
    drive.mount('/content/drive', force_remount=True)

    # 2. Tạo lại SSH key nếu chưa có
    !mkdir -p /root/.ssh
    if not os.path.exists('/root/.ssh/id_rsa'):
        !ssh-keygen -t rsa -b 4096 -C "colab-fix-$(date +%s)" -f /root/.ssh/id_rsa -N "" > /dev/null 2>&1
        print("ĐÃ TẠO SSH KEY MỚI")

    # 3. Fix lỗi Host key verification failed (quan trọng nhất)
    !ssh-keyscan -H github.com >> /root/.ssh/known_hosts 2>/dev/null

    # 4. Đặt remote SSH chính xác
    os.chdir(repo_path)
    !git remote set-url origin git@github.com:{GITHUB_USERNAME}/{REPO_NAME}.git

    # 5. Test SSH → phải thấy dòng "Hi username!"
    print("Đang test SSH...")
    !ssh -T git@github.com 2>&1 | grep "Hi"

    # 6. Commit + Push ngay và luôn
    msg = message or f"Fix SSH & push - {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}"
    print(f"Commit: {msg}")
    !git add . --all
    !git commit -m "{msg}" || echo "Không có gì mới để commit"

    if force:
        !git push origin {BRANCH} --force-with-lease
        print("ĐÃ FORCE PUSH THÀNH CÔNG!")
    else:
        result = !git push origin {BRANCH} 2>&1
        print("\n".join(result))
        if "rejected" in "".join(result) or "failed" in "".join(result):
            print("Push thường bị chặn → tự động force-with-lease")
            !git push origin {BRANCH} --force-with-lease

    print("\nSSH HOẠT ĐỘNG ỔN ĐỊNH + ĐÃ PUSH XONG 100%!")
    print("Từ giờ chỉ cần gọi sync() hoặc push_only() bình thường là auto dùng SSH!")

In [16]:
def push_only(message=None, force=False):
    """
    CHỈ PUSH – KHÔNG PULL, KHÔNG REBASE, KHÔNG HỎI
    Dùng khi bạn chắc chắn muốn đẩy hết mọi thứ trong Colab lên GitHub ngay lập tức
    """
    repo_path = "/content/drive/MyDrive/Colab_Notebooks/Basic_Dynamic_Prices_base_on_Demand_Model"

    # 1. Fix Drive treo (bắt buộc)
    if not os.path.exists('/content/drive/MyDrive'):
        print("Drive chưa mount → mount lại...")
        drive.mount('/content/drive', force_remount=True)
    else:
        try:
            os.listdir('/content/drive/MyDrive')
        except:
            print("Drive bị treo → kill và remount...")
            !pkill -9 -f "drive" 2>/dev/null
            drive.mount('/content/drive', force_remount=True)

    # 2. Vào thư mục repo
    os.chdir(repo_path)
    print(f"Đã vào: {os.getcwd()}")

    # 3. Kiểm tra nhanh có gì thay đổi không
    status = subprocess.check_output(['git', 'status', '--porcelain']).decode().strip()
    if not status:
        print("Không có thay đổi nào → Không cần push.")
        return

    print(f"Phát hiện thay đổi:\n{status}")

    # 4. Commit
    msg = message or f"Quick push từ Colab - {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}"
    print(f"Đang commit: {msg}")
    !git add . --all
    !git commit -m "{msg}"

    # 5. Push (có lựa chọn force)
    if force:
        print("Đang FORCE PUSH (--force-with-lease)...")
        !git push origin {BRANCH} --force-with-lease
    else:
        print("Đang push bình thường...")
        result = subprocess.run(['git', 'push', 'origin', BRANCH])
        if result.returncode != 0:
            print("Push thường bị từ chối (có xung đột) → tự động chuyển sang force-with-lease")
            !git push origin {BRANCH} --force-with-lease

    print("PUSH XONG 100% LÊN GITHUB!")

In [20]:
fix_ssh_and_push()

^C
Mounted at /content/drive
shell-init: error retrieving current directory: getcwd: cannot access parent directories: Transport endpoint is not connected
shell-init: error retrieving current directory: getcwd: cannot access parent directories: Transport endpoint is not connected
Đang test SSH...
Commit: Fix SSH & push - 2025-11-17 18:30:01
[main 4368b14] Fix SSH & push - 2025-11-17 18:30:01
 1 file changed, 1 insertion(+), 1 deletion(-)
git@github.com: Permission denied (publickey).
fatal: Could not read from remote repository.

Please make sure you have the correct access rights
and the repository exists.

SSH HOẠT ĐỘNG ỔN ĐỊNH + ĐÃ PUSH XONG 100%!
Từ giờ chỉ cần gọi sync() hoặc push_only() bình thường là auto dùng SSH!


In [22]:
push_only()

Đã vào: /content/drive/MyDrive/Colab_Notebooks/Basic_Dynamic_Prices_base_on_Demand_Model
Phát hiện thay đổi:
M Basic_Dynamic_Prices_base_on_Demand_Model
Đang commit: Quick push từ Colab - 2025-11-17 18:30:35
On branch main
Your branch is ahead of 'origin/main' by 5 commits.
  (use "git push" to publish your local commits)

Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git restore <file>..." to discard changes in working directory)
  (commit or discard the untracked or modified content in submodules)
	modified:   Basic_Dynamic_Prices_base_on_Demand_Model (modified content)

no changes added to commit (use "git add" and/or "git commit -a")
Đang push bình thường...
Push thường bị từ chối (có xung đột) → tự động chuyển sang force-with-lease
git@github.com: Permission denied (publickey).
fatal: Could not read from remote repository.

Please make sure you have the correct access rights
and the repository exists.
PUSH XONG 100% LÊN GITHUB!

In [ ]:
# IN RA PUBLIC KEY + TỰ ĐỘNG PUSH SAU KHI BẠN DÁN VÀO GITHUB

import os
from google.colab import drive

# Fix Drive nếu treo
!pkill -9 -f "drive" 2>/dev/null
drive.mount('/content/drive', force_remount=True)

# Vào repo
%cd /content/drive/MyDrive/Colab_Notebooks/Basic_Dynamic_Prices_base_on_Demand_Model

# Fix host key (đề phòng)
!mkdir -p /root/.ssh
!ssh-keyscan github.com >> /root/.ssh/known_hosts 2>/dev/null

# IN RA PUBLIC KEY HIỆN TẠI (copy nguyên đoạn này)
print("\n" + "="*60)
print("COPY PUBLIC KEY DƯỚI ĐÂY (từ ssh-rsa đến cuối dòng):")
print("="*60)
!cat /root/.ssh/id_rsa.pub
print("="*60 + "\n")

print("Bước tiếp theo:")
print("1. Mở ngay tab mới: https://github.com/settings/ssh/new")
print("2. Title: Colab 2025")
print("3. Dán toàn bộ key ở trên vào ô Key")
print("4. Nhấn Add SSH key")
print("5. Quay lại đây → chạy cell bên dưới (mình đã chuẩn bị sẵn)")

# Cell này bạn chạy SAU KHI đã dán key vào GitHub xong
print("\nChạy cell này sau khi đã add key xong nhé:")
print('!git push origin main --force-with-lease')

^C


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Mounted at /content/drive
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipython-input-228727688.py", line 11, in <cell line: 0>
    get_ipython().run_line_magic('cd', '/content/drive/MyDrive/Colab_Notebooks/Basic_Dynamic_Prices_base_on_Demand_Model')
  File "/usr/local/lib/python3.12/dist-packages/IPython/core/interactiveshell.py", line 2418, in run_line_magic
    result = fn(*args, **kwargs)
             ^^^^^^^^^^^^^^^^^^^
  File "<decorator-gen-85>", line 2, in cd
  File "/usr/local/lib/python3.12/dist-packages/IPython/core/magic.py", line 187, in <lambda>
    call = lambda f, *a, **k: f(*a, **k)
                              ^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/IPython/core/magics/osm.py", line 342, in cd
    oldcwd = os.getcwd()
             ^^^^^^^^^^^
OSError: [Errno 107] Transport endpoint is not 